In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers

import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.6.0'

# Data Pre-Processing

Open **play_style_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`. 

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves, the last move represents the play style (B[di] in this case)
    
We cropped only the moves to game list as:

In [3]:
df = open('./Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [5]:
#0:所有跟最後一步顏色相同的
#1:所有跟最後一步顏色不同的
#2:標示空地
#3~10:最後8步
#11:周圍黑棋7*7
#12:周圍白棋7*7
def prepare_input(moves):
    x = np.zeros((19,19,13))
    player_color = moves[-1][0] #
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == player_color: #
            x[row,column,0] = 1
            x[row,column,2] = 1
        else: #
            x[row,column,1] = 1
            x[row,column,2] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    
    #倒數8步
    sz = len(moves)
    last1 = sz - 1
    last8 = max(sz - 9, 0)
    for i in range(last1, last8, -1):
        col = coordinates[moves[i][2]]
        row = coordinates[moves[i][3]]
        x[row, col, 11 - (sz - i)] = 1
    
    #周圍7*7
    last_col = coordinates[moves[-1][2]]
    last_row = coordinates[moves[-1][3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 11] = x[i, j, 1]
            x[i, j, 12] = x[i, j, 2]
            
    #列印所有棋盤
#     for i in range(0, 3, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x

In [6]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 26615


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [7]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [8]:
x.shape

(26615, 19, 19, 13)

In [9]:
y.shape

(26615,)

In [10]:
np.bincount(y)

array([8184, 9403, 9028], dtype=int64)

Target is one-hot encoded and loss is changed to `categorical_crossentropy`

In [11]:
y_hot = tf.one_hot(y, depth=3)

Dataset splitting: 90% Training, 10% validation

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.05)

# Training

### Simple DCNN Model:

In [23]:
def create_model():
    inputs = Input(shape=(19, 19, 13))
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(3, activation='softmax', )(outputs)
    model = Model(inputs, outputs)
    opt = Adam(learning_rate=0.00005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [24]:
model = create_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 19, 19, 13)]      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 19, 19, 64)        40832     
_________________________________________________________________
batch_normalization_18 (Batc (None, 19, 19, 64)        256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 19, 19, 64)        200768    
_________________________________________________________________
batch_normalization_19 (Batc (None, 19, 19, 64)        256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 19, 19, 64)        102464    
_________________________________________________________________
batch_normalization_20 (Batc (None, 19, 19, 64)        256 

In [25]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 32,
    epochs = 20,
    validation_data=(x_val, y_val),
)

Epoch 1/20
791/791 [==============================] - 192s 241ms/step - loss: 1.4002 - accuracy: 0.3651 - val_loss: 1.2504 - val_accuracy: 0.4132
Epoch 2/20
791/791 [==============================] - 164s 208ms/step - loss: 1.1542 - accuracy: 0.4236 - val_loss: 1.0775 - val_accuracy: 0.4636
Epoch 3/20
791/791 [==============================] - 165s 208ms/step - loss: 1.0456 - accuracy: 0.4768 - val_loss: 1.0214 - val_accuracy: 0.4914
Epoch 4/20
791/791 [==============================] - 164s 208ms/step - loss: 1.0003 - accuracy: 0.5078 - val_loss: 0.9824 - val_accuracy: 0.5297
Epoch 5/20
791/791 [==============================] - 164s 208ms/step - loss: 0.9742 - accuracy: 0.5281 - val_loss: 0.9707 - val_accuracy: 0.5139
Epoch 6/20
791/791 [==============================] - 164s 208ms/step - loss: 0.9618 - accuracy: 0.5359 - val_loss: 0.9638 - val_accuracy: 0.5177
Epoch 7/20
791/791 [==============================] - 165s 208ms/step - loss: 0.9541 - accuracy: 0.5406 - val_loss: 0.9524 -

In [22]:
model.save('./models/model_playstyle_inputfor13_batch32_p4.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!